## 简答题

1. TensorFlow 是否可以简单替代 NumPy？两者之间的主要区别是什么？

2. 使用 `tf.range(10)` 和 `tf.constant(np.arange(10))` 是否会得到相同的结果？

3. 可以通过编写函数或继承 `tf.keras.losses.Loss` 来定义自定义损失函数。两种方法分别应该在什么时候使用？

4. 可以直接在函数中定义自定义指标或采用 `tf.keras.metrics.Metric` 子类。两种方法分别应该在什么时候使用？

5. 什么时候应该自定义层而不是自定义模型？

6. 有哪些示例需要编写自定义训练循环？

7. 自定义 Keras 组件中可以包含任意 Python 代码，还是必须转换为 TF 函数？

8. 如果要将函数转换为 TF 函数，应避免哪些主要模式？

9. 何时需要创建动态 Keras 模型？ 如何动态创建Keras模型？为什么不是所有模型都动态化？


## 编程题

1. 实现一个执行层归一化的自定义层：
    - a. `build()` 方法应定义两个可训练的权重 α 和 β，它们的形状均为 `input_shape[-1:]`，数据类型为 `tf.float32`。α 应该用 1 初始化，而 β 必须用 0 初始化。
    - b. `call()` 方法应计算每个实例特征的均值和标准差。为此，可以使用 `tf.nn.moments(inputs, axes=-1, keepdims=True)`，它返回同一实例的均值 μ 和方差 σ²（计算方差的平方根便可获得标准差）。然后，该函数应计算并返回
      $$
      \alpha \otimes \frac{(X-\mu)}{(\sigma+\epsilon)} + \beta
      $$
      其中 ε 是表示项精度的一个常量（避免被零除的小常数，例如 0.001）,$\otimes$表示逐个元素相乘
    - c. 确保自定义层产生与tf.keras.layers.LayerNormalization层相同（或几乎相同）的输出。

2. 使用自定义训练循环训练模型来处理Fashion MNIST数据集（13_神经网络介绍 里用的数据集）：

    - a.显示每个轮次、迭代、平均训练损失和每个轮次的平均精度（在每次迭代中更新），以及每个轮次结束时的验证损失和精度。
    - b.尝试对上面的层和下面的层使用具有不同学习率的不同优化器。

In [1]:
import tensorflow as tf

class CustomLayerNormalization(tf.keras.layers.Layer):
    def __init__(self, epsilon=0.001, **kwargs):
        super(CustomLayerNormalization, self).__init__(**kwargs)
        self.epsilon = epsilon

    def build(self, input_shape):
        # 创建可训练参数 alpha (gamma) 和 beta，形状为最后一个维度
        shape = input_shape[-1:]
        self.alpha = self.add_weight(
            name='alpha',
            shape=shape,
            initializer='ones',  # α 初始化为 1
            trainable=True,
            dtype=tf.float32
        )
        self.beta = self.add_weight(
            name='beta',
            shape=shape,
            initializer='zeros',  # β 初始化为 0
            trainable=True,
            dtype=tf.float32
        )
        super(CustomLayerNormalization, self).build(input_shape)

    def call(self, inputs):
        # 计算均值和方差，沿最后一个轴（特征轴），保持维度
        mean, variance = tf.nn.moments(inputs, axes=-1, keepdims=True)
        std = tf.sqrt(variance + self.epsilon)  # 标准差，加 epsilon 防止除零

        # 归一化: (X - μ) / (σ + ε)
        normalized = (inputs - mean) / std

        # 缩放和平移: α ⊗ normalized + β
        return self.alpha * normalized + self.beta

    def get_config(self):
        config = super(CustomLayerNormalization, self).get_config()
        config.update({'epsilon': self.epsilon})
        return config

In [2]:
import numpy as np

# 创建测试输入
np.random.seed(42)
test_input = np.random.randn(4, 10, 8).astype(np.float32)  # (batch, time, features)

# 实例化两个层
custom_ln = CustomLayerNormalization()
official_ln = tf.keras.layers.LayerNormalization(axis=-1, epsilon=0.001)

# 构建权重（确保两者使用相同初始化）
_ = custom_ln(test_input)
_ = official_ln(test_input)

# 手动设置官方层的 gamma 和 beta 与自定义层一致（用于精确比较）
official_ln.gamma.assign(custom_ln.alpha)
official_ln.beta.assign(custom_ln.beta)

# 前向传播
output_custom = custom_ln(test_input)
output_official = official_ln(test_input)

# 比较差异
diff = tf.reduce_max(tf.abs(output_custom - output_official))
print(f"最大绝对误差: {diff:.6f}")
assert diff < 1e-6, "输出不一致！"
print("✅ 自定义层与官方 LayerNormalization 输出一致！")

最大绝对误差: 0.000000
✅ 自定义层与官方 LayerNormalization 输出一致！


In [4]:
import tensorflow as tf
import numpy as np
from sklearn.metrics import accuracy_score

# ========== 1. 加载并预处理 Fashion MNIST ==========
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
x_train = x_train.astype(np.float32) / 255.0
x_test = x_test.astype(np.float32) / 255.0
x_train = x_train[..., tf.newaxis]  # (60000, 28, 28, 1)
x_test = x_test[..., tf.newaxis]    # (10000, 28, 28, 1)
y_train = y_train.astype(np.int64)
y_test = y_test.astype(np.int64)



# 构建模型
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(28, 28, 1), name='conv1'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(64, 3, activation='relu', name='conv2'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu', name='dense1'),
    CustomLayerNormalization(name='custom_ln'),
    tf.keras.layers.Dense(10, name='classifier')  # 输出层，无激活（配合 SparseCategoricalCrossentropy(from_logits=True)）
])

# ========== 3. 定义不同层的优化器 ==========
# 分组参数
conv_vars = model.get_layer('conv1').trainable_variables + model.get_layer('conv2').trainable_variables
dense_vars = model.get_layer('dense1').trainable_variables + model.get_layer('custom_ln').trainable_variables
classifier_vars = model.get_layer('classifier').trainable_variables

# 为不同组创建不同优化器
optimizer_conv = tf.keras.optimizers.Adam(learning_rate=1e-4)   # 底层卷积：小学习率
optimizer_dense = tf.keras.optimizers.Adam(learning_rate=1e-3)  # 中间层：中等学习率
optimizer_classifier = tf.keras.optimizers.SGD(learning_rate=0.01)  # 分类层：大学习率 + SGD

# 损失函数和指标
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
train_acc_metric = tf.keras.metrics.SparseCategoricalAccuracy()
val_acc_metric = tf.keras.metrics.SparseCategoricalAccuracy()

# ========== 4. 自定义训练步骤 ==========
@tf.function
def train_step(x, y):
    with tf.GradientTape(persistent=True) as tape:
        logits = model(x, training=True)
        loss = loss_fn(y, logits)

    # 分别计算不同组的梯度并应用
    grads_conv = tape.gradient(loss, conv_vars)
    optimizer_conv.apply_gradients(zip(grads_conv, conv_vars))

    grads_dense = tape.gradient(loss, dense_vars)
    optimizer_dense.apply_gradients(zip(grads_dense, dense_vars))

    grads_classifier = tape.gradient(loss, classifier_vars)
    optimizer_classifier.apply_gradients(zip(grads_classifier, classifier_vars))

    del tape  # 手动释放 persistent tape
    return loss, logits

@tf.function
def val_step(x, y):
    logits = model(x, training=False)
    loss = loss_fn(y, logits)
    return loss, logits

# ========== 5. 数据批处理 ==========
batch_size = 32
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(batch_size)
val_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size)

# ========== 6. 训练循环 ==========
epochs = 3

for epoch in range(epochs):
    print(f"\nEpoch {epoch + 1}/{epochs}")
    train_loss_sum = 0.0
    num_batches = 0

    # 重置训练精度
    train_acc_metric.reset_states()

    for step, (x_batch, y_batch) in enumerate(train_dataset):
        loss, logits = train_step(x_batch, y_batch)
        train_loss_sum += loss
        num_batches += 1

        # 更新并显示当前 batch 的精度
        train_acc_metric.update_state(y_batch, logits)
        current_acc = train_acc_metric.result().numpy()

        if step % 50 == 0:
            print(f"  Step {step:4d} | Loss: {loss:.4f} | Accuracy: {current_acc:.4f}")

    # 计算平均训练损失和精度
    avg_train_loss = train_loss_sum / num_batches
    avg_train_acc = train_acc_metric.result().numpy()

    # 验证阶段
    val_loss_sum = 0.0
    val_num_batches = 0
    val_acc_metric.reset_states()

    for x_batch, y_batch in val_dataset:
        loss, logits = val_step(x_batch, y_batch)
        val_loss_sum += loss
        val_num_batches += 1
        val_acc_metric.update_state(y_batch, logits)

    avg_val_loss = val_loss_sum / val_num_batches
    avg_val_acc = val_acc_metric.result().numpy()

    # 显示轮次汇总
    print(f"Epoch {epoch + 1} Summary:")
    print(f"  Train Loss: {avg_train_loss:.4f} | Train Accuracy: {avg_train_acc:.4f}")
    print(f"  Val Loss:   {avg_val_loss:.4f} | Val Accuracy:   {avg_val_acc:.4f}")


Epoch 1/3
  Step    0 | Loss: 2.6239 | Accuracy: 0.1562
  Step   50 | Loss: 0.7415 | Accuracy: 0.6330
  Step  100 | Loss: 0.5395 | Accuracy: 0.6903
  Step  150 | Loss: 0.3453 | Accuracy: 0.7171
  Step  200 | Loss: 0.4961 | Accuracy: 0.7387
  Step  250 | Loss: 0.3771 | Accuracy: 0.7542
  Step  300 | Loss: 0.7706 | Accuracy: 0.7659
  Step  350 | Loss: 0.3381 | Accuracy: 0.7763
  Step  400 | Loss: 0.5252 | Accuracy: 0.7825
  Step  450 | Loss: 0.3935 | Accuracy: 0.7895
  Step  500 | Loss: 0.4805 | Accuracy: 0.7940
  Step  550 | Loss: 0.4810 | Accuracy: 0.7989
  Step  600 | Loss: 0.1966 | Accuracy: 0.8033
  Step  650 | Loss: 0.4827 | Accuracy: 0.8081
  Step  700 | Loss: 0.3986 | Accuracy: 0.8103
  Step  750 | Loss: 0.2668 | Accuracy: 0.8136
  Step  800 | Loss: 0.4087 | Accuracy: 0.8164
  Step  850 | Loss: 0.6098 | Accuracy: 0.8196
  Step  900 | Loss: 0.2995 | Accuracy: 0.8220
  Step  950 | Loss: 0.2164 | Accuracy: 0.8239
  Step 1000 | Loss: 0.3198 | Accuracy: 0.8255
  Step 1050 | Loss: 0.3